---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [17]:
def graph_identification():
    
    # Your Code Here
    
    alg_tot=[]
    for G in P1_Graphs:
        ac=nx.average_clustering(G)
        #print(ac)
        aspl=nx.average_shortest_path_length(G)
        #print(aspl)

        if ac<0.01:
            alg='PA'
        elif aspl<5:
            alg='SW_H'
        elif aspl>5:
            alg='SW_L'

        alg_tot.append(alg)
        #print(alg)
        
    return alg_tot# Your Answer Here

#graph_identification()

['SW_H', 'SW_L', 'SW_L', 'SW_H', 'PA']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [18]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [82]:
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    
    #Adding node attributes
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['centrality']=pd.Series(nx.degree_centrality(G))
    df['closeness_centrality']=pd.Series(nx.closeness_centrality(G))
    df['betweenness_centrality']=pd.Series(nx.betweenness_centrality(G))
    
    #splitting into train & test df
    train_df=df[df['ManagementSalary'].notnull()]
    test_df=df[df['ManagementSalary'].isnull()]
    
    train_X=train_df.loc[:,train_df.columns != 'ManagementSalary']
    train_y=train_df.loc[:,'ManagementSalary']
    test_X=test_df.loc[:,test_df.columns != 'ManagementSalary']
    
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=0.05,class_weight='balanced')
    clf.fit(train_X, train_y)
    
    test_y_pred=pd.Series(clf.predict(test_X))
    test_y_pred.index=test_X.index
    test_y_pred
    
    return test_y_pred# Your Answer Here

#salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [63]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [91]:
def new_connections_predictions():
    
    # Your Code Here
    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbors'] = future_connections.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['resource_allocation_index'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    #df['adamic_adar_index'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index,community='Department')]
    future_connections['ra_index_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index,community='Department')]

    train_df=future_connections[future_connections['Future Connection'].notnull()]
    test_df=future_connections[future_connections['Future Connection'].isnull()]
    
    train_X=train_df.loc[:,train_df.columns != 'Future Connection']
    train_y=train_df.loc[:,'Future Connection']
    test_X=test_df.loc[:,test_df.columns != 'Future Connection']
    
    
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=0.05,class_weight='balanced')
    clf.fit(train_X, train_y)
    
    
    test_y_prob=pd.Series(clf.predict_proba(test_X)[:,1])
    test_y_prob.index=test_X.index
    
    return test_y_prob# Your Answer Here

#new_connections_predictions()

(107, 348)    0.252098
(542, 751)    0.128238
(20, 426)     0.966149
(50, 989)     0.128238
(942, 986)    0.128836
(324, 857)    0.128238
(13, 710)     0.711690
(19, 271)     0.731627
(319, 878)    0.128238
(659, 707)    0.128238
(49, 843)     0.128391
(208, 893)    0.128238
(377, 469)    0.149738
(405, 999)    0.163885
(129, 740)    0.142335
(292, 618)    0.371814
(239, 689)    0.128238
(359, 373)    0.144583
(53, 523)     0.667103
(276, 984)    0.128238
(202, 997)    0.128391
(604, 619)    0.326472
(270, 911)    0.128238
(261, 481)    0.538682
(200, 450)    0.971569
(213, 634)    0.128238
(644, 735)    0.346058
(346, 553)    0.128238
(521, 738)    0.127930
(422, 953)    0.163512
                ...   
(672, 848)    0.128238
(28, 127)     0.970094
(202, 661)    0.128238
(54, 195)     0.969983
(295, 864)    0.128238
(814, 936)    0.128238
(839, 874)    0.128836
(139, 843)    0.128238
(461, 544)    0.135454
(68, 487)     0.134700
(622, 932)    0.128238
(504, 936)    0.166987
(479, 528) 